**Table of contents**<a id='toc0_'></a>    
- [Unsupervised Learning](#toc1_)    
  - [Load breast cancer data](#toc1_1_)    
  - [Dimensionality reduction](#toc1_2_)    
    - [PCA (Principal Component Analysis)](#toc1_2_1_)    
    - [How to choose the number of principal components?](#toc1_2_2_)    
  - [Clustering](#toc1_3_)    
    - [Centroid-based clustering - K-Means](#toc1_3_1_)    
      - [Elbow method - choosing the # of clusters](#toc1_3_1_1_)    
    - [Agglomerative/Hierarchical/Connectivity-based clustering](#toc1_3_2_)    
      - [Ward linkage](#toc1_3_2_1_)    
      - [Complete linkage](#toc1_3_2_2_)    
      - [Single linkage](#toc1_3_2_3_)    
    - [Where is agglomerative clustering better than centroid-based clustering?](#toc1_3_3_)    
    - [Density-based clustering - DBSCAN (Density-based spatial clustering of applications with noise)](#toc1_3_4_)    
  - [Clustering metrics](#toc1_4_)    
- [Resources](#toc2_)    
- [References](#toc3_)    
- [Acknowledgements](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Unsupervised Learning](#toc0_)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'simple_white'

import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn import cluster, datasets
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from yellowbrick.cluster import KElbowVisualizer

## <a id='toc1_1_'></a>[Load breast cancer data](#toc0_)

In [ ]:
# Unsupervised Learning Class
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print(cancer['DESCR'])

In [ ]:
X = cancer['data']
y = cancer['target']
display(X.shape)
display(y.shape)

In [ ]:
# let's visualize all the data in a dataframe
data = pd.DataFrame(X, columns=cancer['feature_names'])
data['label'] = y
data.head()

## <a id='toc1_2_'></a>[Dimensionality reduction](#toc0_)

### <a id='toc1_2_1_'></a>[PCA (Principal Component Analysis)](#toc0_)

Principal Component Analysis (PCA) is a statistical technique used for dimensionality reduction. It transforms the data into a new coordinate system such that the greatest variance by any projection of the data comes to lie on the first coordinate (called the first principal component), the second greatest variance on the second coordinate, and so on.

![](https://imgs.search.brave.com/1u8pddu4FRUlSAuxj0i992lYN__bYby0JDUEqF2abEM/rs:fit:860:0:0/g:ce/aHR0cHM6Ly9icmFk/bGV5Ym9laG1rZS5n/aXRodWIuaW8vSE9N/TC8xNS1wY2FfZmls/ZXMvZmlndXJlLWh0/bWwvY3JlYXRlLXBj/YS1pbWFnZS0xLnBu/Zw)  
(Source: [Hands-on Machine Learning with R, Bradley Boehmke](https://bradleyboehmke.github.io/HOML/pca.html))

For this dataset, we're not looking to predict whether or not a patient has a malignant/benign tumour, but if we can create 2 clusters corresponding to the most similar tumours. However, we think that the malignant tumours are significantly different from benign tumours, which is why we're looking to see if we can separate clusters based on the tumour type.

In [ ]:
#check differences between features and labels
plot_options, charts = plt.subplots(5, 6, figsize=(20, 7))
malignant = data[data['label'] == 1]
benign = data[data['label'] == 0]
#ravel flattens the array so we don't need 2 indexes
charts_1d = charts.ravel()
for i in range(30):
  charts_1d[i].hist(malignant.iloc[:, i], bins=50, alpha=.5)
  charts_1d[i].hist(benign.iloc[:, i], bins=50, alpha=.5)
  charts_1d[i].set_title(data.columns[i])
  plot_options.tight_layout()

Ideally, we'd want to see if we can cluster the tumours based on multiple features (like we did for the iris dataset some time ago) but in this case we have >30 labels, which makes a pairplot take ages to run...

In [ ]:
# sns.pairplot(data, hue = 'label')

We can still look to see if we can separate clusters based on 2 features at a time:

In [ ]:
# Cluster boundary for features 1, 29
fig = px.scatter(x=data.iloc[:, 1], y=data.iloc[:, 29], color=y, opacity=0.5)
fig.update_layout(xaxis_title=data.columns[1], yaxis_title=data.columns[29], height=600, width=600)
fig.show()

# Cluster boundary for features 28, 29
fig = px.scatter(x=data.iloc[:, 28], y=data.iloc[:, 29], color=y, opacity=0.5)
fig.update_layout(xaxis_title=data.columns[28], yaxis_title=data.columns[29], height=600, width=600)
fig.show()

In [ ]:
# Scale data
scaler = MinMaxScaler()
scaler.fit(cancer['data'])

X_scaled = scaler.transform(cancer['data'])
pd.DataFrame(X_scaled, columns=cancer.feature_names)

Why do we need to perform scaling before running the PCA?

In [ ]:
# create the PCA object
# the number of components chosen will be the new number of features!
pca = PCA(n_components=3)
# fit the PCA model to breast cancer data
pca.fit(X_scaled)
# it's like we have three new axis (those defined by the PCA principal components)
X_pca = pca.transform(X_scaled)
pd.DataFrame(X_pca, columns=['PCA_1', 'PCA_2', 'PCA_3'])

In [ ]:
display(X.shape)
display(X_pca.shape)

In [ ]:
fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=y, opacity=0.5)
fig.update_layout(xaxis_title='1st principal component', yaxis_title='2nd principal component', height=600, width=600)
fig.show()

fig = px.scatter(x=X_pca[:, 1], y=X_pca[:, 2], color=y, opacity=0.5)
fig.update_layout(xaxis_title='2nd principal component', yaxis_title='3rd principal component', height=600, width=600)
fig.show()

fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 2], color=y, opacity=0.5)
fig.update_layout(xaxis_title='1st principal component', yaxis_title='3rd principal component', height=600, width=600)
fig.show()

### <a id='toc1_2_2_'></a>[How to choose the number of principal components?](#toc0_)

In [ ]:
# Each PCA feature has a little bit of each original feture
# The PCA tells you how much of the original features each new feature contains
pd.DataFrame(pca.components_, columns=cancer.feature_names)

In [ ]:
# Matplotlib
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1, 2], ["First component", "Second component","Third component"])
plt.colorbar()
plt.xticks(range(len(data.columns)), data.columns, rotation=90)
plt.show()

> Mathematically, the new features (principal components) are a LINEAR COMBINATION of the previous (old) features and the weights of each of them are represented in the diagram above.

## <a id='toc1_3_'></a>[Clustering](#toc0_)

### <a id='toc1_3_1_'></a>[Centroid-based clustering - K-Means](#toc0_)

![](https://cdn.sanity.io/images/kuana2sp/production/4a7a2b92082d482c56e0c6396064ca23074168ac-1020x752.png?w=1080&fit=max&auto=format)  
(Source: [Getting started with k-means clustering in Python, Dr J Rogel-Salazar](https://domino.ai/blog/getting-started-with-k-means-clustering-in-python))

In [ ]:
# Creating 3 clusters artificially to illustrate the algorithm
n_samples = 1500
X, y = datasets.make_blobs(n_samples=n_samples, centers=3, cluster_std=0.7, n_features=2, random_state=0)

Review initial data:

In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

Run clustering with:

a) k = 3

In [ ]:
kmeans = cluster.KMeans(n_clusters=3)
kmeans.fit(X)
pred = kmeans.predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

b) k = 2

In [ ]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(X)
pred = kmeans.predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

c) k = 4

In [ ]:
kmeans = cluster.KMeans(n_clusters=4)
kmeans.fit(X)
pred = kmeans.predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

#### <a id='toc1_3_1_1_'></a>[Elbow method - choosing the # of clusters](#toc0_)

We can select a number of clusters using the elbow heuristic:

In [ ]:
model = cluster.KMeans()
visualizer = KElbowVisualizer(model, k=(2,20))
visualizer.fit(X)
visualizer.poof()

Technically, any k between 4 and 8 would be good enough for this problem. Remember, we're not looking to minimize the error in this case!

In [ ]:
model = cluster.KMeans()
visualizer = KElbowVisualizer(model, k=(1495,1499))
visualizer.fit(X)
visualizer.poof()

If we try to minimize the error, we end up with an unusable number of clusters, which defeats the purpose of running the algorithm to begin with.

### <a id='toc1_3_2_'></a>[Agglomerative/Hierarchical/Connectivity-based clustering](#toc0_)

![](https://media.geeksforgeeks.org/wp-content/uploads/20200204181551/Untitled-Diagram71.png)  
(Source: [Hierarchical Clustering in Machine Learning, Geeks 4 Geeks](https://www.geeksforgeeks.org/ml-hierarchical-clustering-agglomerative-and-divisive-clustering/))

#### <a id='toc1_3_2_1_'></a>[Ward linkage](#toc0_)

In [ ]:
# ward linkage tends to produce relatively equally sized clusters
agglomerative = cluster.AgglomerativeClustering(n_clusters=3, linkage='ward')
pred = agglomerative.fit_predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

#### <a id='toc1_3_2_2_'></a>[Complete linkage](#toc0_)

In [ ]:
# complete linkage penalizes heavily outliers
agglomerative = cluster.AgglomerativeClustering(n_clusters=3,linkage='complete')
pred = agglomerative.fit_predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

#### <a id='toc1_3_2_3_'></a>[Single linkage](#toc0_)

In [ ]:
# different algorithms are good for different applications
agglomerative = cluster.AgglomerativeClustering(n_clusters=3, linkage='single')
pred = agglomerative.fit_predict(X)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

### <a id='toc1_3_3_'></a>[Where is agglomerative clustering better than centroid-based clustering?](#toc0_)

In [ ]:
# Create moon-like clusters instead
n_samples = 1500
X, y = datasets.make_moons(n_samples=n_samples, noise=.05)

X = StandardScaler().fit_transform(X)

# Run K-Means on non-radial clusters
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(X)
y1_pred = kmeans.predict(X)

# Run Agglomerative clustering on non-radial clusters
single = cluster.AgglomerativeClustering(n_clusters=2, linkage='single')
y2_pred = single.fit_predict(X)

# Review results
options, charts = plt.subplots(1, 2, figsize=(12, 6))
colors = np.array(['blue', 'red'])
charts[0].scatter(X[:, 0], X[:, 1], color=colors[y1_pred])
charts[1].scatter(X[:, 0], X[:, 1], color=colors[y2_pred])
plt.show()

### <a id='toc1_3_4_'></a>[Density-based clustering - DBSCAN (Density-based spatial clustering of applications with noise)](#toc0_)

> In density-based clustering, clusters are defined as areas of higher density than the remainder of the data set. Objects in sparse areas – that are required to separate clusters – are usually considered to be noise and border points.

In [ ]:
n_samples = 1500
X, y = datasets.make_moons(n_samples=n_samples, noise=.05)

In [ ]:
dbs = cluster.DBSCAN(eps=0.1, min_samples=5) #change maximum distance and see efect
# eps is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
# min_samples is the number of samples in a neighborhood for a point to be considered as a core point.

pred = dbs.fit_predict(X)
print(pred)
set(pred)

In [ ]:

plt.figure(figsize=(12, 6))

colors = np.array(['blue', 'red', 'black', 'green', 'yellow'])
plt.scatter(X[:, 0], X[:, 1], color=colors[pred])

plt.show()

## <a id='toc1_4_'></a>[Clustering metrics](#toc0_)

**Silhouette Score** [$^{[3]}$](https://www.educative.io/answers/what-is-silhouette-score)

> Silhouette Score is a tool for assessing the appropriateness of clustering results by providing a quantitative measure of how well-defined and distinct the clusters are. The Silhouette Score quantifies **how well a data point fits into its assigned cluster** and **how distinct it is from other clusters** 

Silhouette Score = $\frac{(b - a)}{max(a, b)}$

In [ ]:
# Re-create clusters
n_samples = 1500
X, y = datasets.make_blobs(n_samples=n_samples, centers=3, cluster_std=0.7, n_features=2, random_state=0)

kmeans = cluster.KMeans(n_clusters=3)
kmeans.fit(X)
pred = kmeans.predict(X)

print("Model 1 Silhouette Score: {}".format(silhouette_score(X, pred, metric='euclidean')))

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

In [ ]:
agglomerative = cluster.AgglomerativeClustering(n_clusters=3, linkage='complete')
pred = agglomerative.fit_predict(X)

print("Model 2 Silhouette Score: {}".format(silhouette_score(X, pred, metric='euclidean')))

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

In [ ]:
agglomerative = cluster.AgglomerativeClustering(n_clusters=3, linkage='single')
pred = agglomerative.fit_predict(X)

print("Model 3 Silhouette Score: {}".format(silhouette_score(X, pred, metric='euclidean')))

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=pred, opacity=0.5)
fig.update_layout(height=600, width=600)
fig.show()

> The Silhouette Score ranges from -1 to +1. Here is how to interpret the value: [$^{[3]}$](https://www.educative.io/answers/what-is-silhouette-score)

**Negative**
> A negative score indicates that the **data point is likely assigned to the wrong cluster**, as its distance to its assigned cluster’s points is greater than its distance to the nearest neighboring cluster’s points.

**Close to 0**
> A score close to 0 implies that the **data point is on or very close to the decision boundary between two clusters**. It indicates that the clustering is not well-defined and can be ambiguous.

**Positive**
> A positive score indicates that the **data point is appropriately clustered**, and **its distance to its assigned cluster’s points is smaller than its distance to the nearest neighboring cluster’s points**. A score close to +1 suggests that the data point is well-clustered and distinctly separated from other clusters. It is a strong indication of a meaningful clustering result.

Where does the silhouette score fail to be a good measure?

In [ ]:
n_samples = 1500
X, y = datasets.make_moons(n_samples=n_samples, noise=.05)
X = StandardScaler().fit_transform(X)

kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(X)
y1_pred = kmeans.predict(X)


single = cluster.AgglomerativeClustering(n_clusters=2, linkage='single')
y2_pred = single.fit_predict(X)


options, charts = plt.subplots(1, 2, figsize=(12, 6))
colors = np.array(['blue', 'red'])
charts[0].scatter(X[:, 0], X[:, 1], color=colors[y1_pred])
charts[1].scatter(X[:, 0], X[:, 1], color=colors[y2_pred])
plt.show()

print("Model 1 Silhouette Score: {}".format(silhouette_score(X, y1_pred)))
print("Model 2 Silhouette Score: {}".format(silhouette_score(X, y2_pred)))

# <a id='toc2_'></a>[Resources](#toc0_)

PCA by StatQuest:
- [Main ideas (5 min)](https://www.youtube.com/watch?v=HMOI_lkzW08)
- [Step-by-step (22 min)](https://www.youtube.com/watch?v=FgakZw6K1QQ)
- [Practical Tips (8 min)](https://www.youtube.com/watch?v=oRvgq966yZg)  

[t-SNEs by StatQuest (12 min)](https://www.youtube.com/watch?v=NEaUSP4YerM)

# <a id='toc3_'></a>[References](#toc0_)

[1] [What is Silhouette Score, Educative IO](https://www.educative.io/answers/what-is-silhouette-score)

# <a id='toc4_'></a>[Acknowledgements](#toc0_)

Thank you, David Henriques, for your awesome lesson structure and content!